<div class="alert alert-danger">
    This session will be recorded.
</div>

# "Isochrones" with OSM


How far can you travel on foot in 15 minutes?

<img src="images/isos.png" width=800>



## What is open street map?

![osm](images/OSM.png)

OpenStreetMap (OSM) is a global collaborative (crowd-sourced) dataset and project that aims at creating a free editable map of the world containing a lot of information about our environment. It contains data for example about streets, buildings, different services, and landuse to mention a few. You can view the map at www.openstreetmap.org. You can also sign up as a contributor if you want to edit the map. More details about OpenStreetMap and its contents are available in the OpenStreetMap Wiki.

OSM has a large userbase with more than 7 million users and over a million contributers that update actively the OSM database with 3 million changesets per day. In total OSM contains 6.5 billion nodes and counting! (stats from January 2021).

OpenStreetMap is used not only for integrating the OSM maps as background maps to visualizations or online maps, but also for many other purposes such as routing, geocoding, education, and research. OSM is also widely used for humanitarian response e.g. in crisis areas (e.g. after natural disasters) and for fostering economic development. Read more about humanitarian projects that use OSM data from the [Humanitarian OpenStreetMap Team (HOTOSM) website](https://www.hotosm.org/).


- https://www.openstreetmap.org/
- [OSM stats](https://wiki.openstreetmap.org/wiki/Stats)

## OSMnx

<a href="https://www.spur.org/publications/urbanist-article/2012-11-09/grand-reductions-10-diagrams-changed-city-planning" target="_blank"><img src="https://www.spur.org/sites/default/files/migrated/images/F22.jpg" width=400></a>

[[Image source]](https://www.spur.org/publications/urbanist-article/2012-11-09/grand-reductions-10-diagrams-changed-city-planning)

Allan Jacobs’ seminal treatise Great Streets (1993) takes figure-ground analysis to a new level, showing 50 one-mile-square maps of cities around the world, all drawn to the same scale.


This week we will explore a Python module called OSMnx that can be used to retrieve, construct, analyze, and visualize street networks from OpenStreetMap, and also retrieve data about Points of Interest such as restaurants, schools, and lots of different kind of services. It is also easy to conduct network routing based on walking, cycling or driving by combining OSMnx functionalities with a package called NetworkX.

<img src="https://i2.wp.com/geoffboeing.com/wp-content/uploads/2017/04/square-mile-street-networks.jpg?resize=768%2C617&ssl=1" width=400>

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  

## What's an isochrone map?

<img src="https://www.tandfonline.com/na101/home/literatum/publisher/tandf/journals/content/cppr20/2017/cppr20.v032.i04/02697459.2017.1329487/20171208/images/large/cppr_a_1329487_f0001_oc.jpeg" width=800>

- [Isochrone Mapping of Urban Transport: Car-dependency, Mode-choice and Design Research](https://www.tandfonline.com/doi/figure/10.1080/02697459.2017.1329487?scroll=top&needAccess=true)


## Import the libraries

In [ ]:
# for spatial data
import geopandas as gpd

# for plotting
import matplotlib.pyplot as plt

# for network analysis
import networkx as nx

# for street network analysis
import osmnx as ox

# for basemaps
import contextily as ctx

## Download and prep the street network

When fetching network data from OpenStreetMap using `OSMnx`, it is possible to define the type of street network using the `network_type` parameter (options: `drive`, `walk` and `bike`). Below, we define the parameters that will determine the results of this lab.

There are several methods to bring in Open Street Map data using OSMNX. Choose one that best suits your needs:

Query type | Example scenario| OSMNX command
---|---|---
You have an address and specified distance buffer | 1 kilometer buffer from UCLA | [osmnx.graph.graph_from_address](https://osmnx.readthedocs.io/en/stable/osmnx.html?highlight=graph_from_place#osmnx.graph.graph_from_address)
You have a specific lat/lon coordinate pair and specified distance buffer| 1 kilometer buffer from `[34.12,-118.55]` | [osmnx.graph.graph_from_point](https://osmnx.readthedocs.io/en/stable/osmnx.html?highlight=graph_from_place#osmnx.graph.graph_from_point)
You have a general location or address that has defined boundaries | a neighborhood, zipcode, village, or small city | [osmnx.graph.graph_from_place](https://osmnx.readthedocs.io/en/stable/osmnx.html?highlight=graph_from_place#osmnx.graph.graph_from_place)


<div class="alert alert-danger">
<h1>Warning!</h1>
    Be very careful about what you enter in the "place" variable below. The larger the place, the more data it will download from OpenStreetMap, which can result in overloading your JupyterHub's capacity. 
</div>

In [ ]:
# configure the place, network type, trip times, and travel speed
address = 'Boyle Heights, Los Angeles, CA, USA'
network_type = 'walk'
trip_times = [5, 10, 15, 20] #in minutes
meters_per_minute = 75 # travel distance per minute

In [ ]:
%%time
# %%time is a magic command to see how long it takes this cell to run 

# download the street network
G = ox.graph_from_address(address, network_type=network_type, dist = 2000)

In [ ]:
# what is G?
type(G)

- [Networkx multidigraph](https://networkx.org/documentation/stable/reference/classes/multidigraph.html)

Quick plot to visualize data.

- [plot_graph](https://osmnx.readthedocs.io/en/stable/osmnx.html?highlight=plot_graph#osmnx.plot.plot_graph)

In [ ]:
# quick plot using oxmnx
fig, ax = ox.plot_graph(G,figsize=(12,12))

## Project to web mercator

![projections](https://www.esri.com/arcgis-blog/wp-content/uploads/2022/02/grid2.png)

In order to conduct spatial analysis, it is recommended to use a projected coordinate system, rather than a geographic coordinate system (which uses angular measurements). Here is a [blog post from ESRI](https://www.esri.com/arcgis-blog/products/arcgis-pro/mapping/gcs_vs_pcs/) that describes the differences between the two.

In [ ]:
# project our network data to Web Mercator (measurements are in meters)
G = ox.project_graph(G, to_crs='epsg:3857')

## Convert edges and nodes to geodataframes

OSMnx provides a convenient function `graph_to_gdfs()` that can convert the graph into two separate GeoDataFrames where the first one contains the information about the nodes and the second one about the edge.

Let’s extract the nodes and edges from the graph as GeoDataFrames:

In [ ]:
# convert nodes and edges to geodataframes
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

In [ ]:
gdf_nodes.sample(10)

In [ ]:
gdf_nodes

In [ ]:
gdf_nodes.plot(figsize=(10,10))

In [ ]:
gdf_edges.head()

In [ ]:
gdf_edges.plot(figsize=(10,10))

There are many columns in our GeoDataFrame. Most of the columns are fairly self-explanatory but the following table describes all of them.

Most of the attributes come directly from the OpenStreetMap, however, columns `u` and `v` are Networkx specific ids. You can click on the links to get more information about each attribute:


| Column                                                     | Description                 | Data type         |
|------------------------------------------------------------|-----------------------------|-------------------|
| [bridge](http://wiki.openstreetmap.org/wiki/Key:bridge)    | Bridge feature              | boolean           |
| geometry                                                   | Geometry of the feature     | Shapely.geometry  |
| [highway](http://wiki.openstreetmap.org/wiki/Key:highway)  | Tag for roads (road type)   | str / list        |
| [lanes](http://wiki.openstreetmap.org/wiki/Key:lanes)      | Number of lanes             | int (or nan)      |
| [length](http://wiki.openstreetmap.org/wiki/Key:length)    | Length of feature (meters)  | float             |
| [maxspeed](http://wiki.openstreetmap.org/wiki/Key:maxspeed)| maximum legal speed limit   | int /list         |
| [name](http://wiki.openstreetmap.org/wiki/Key:name)        | Name of the (street) element| str (or nan)      |
| [oneway](http://wiki.openstreetmap.org/wiki/Key:oneway)    | One way road                | boolean           |
| [osmid](http://wiki.openstreetmap.org/wiki/Node)           | Unique ids for the element  | list              |
| [u](http://ow.ly/bV8n30h7Ufm)                              | The first node of edge      | int               |
| [v](http://ow.ly/bV8n30h7Ufm)                              | The last node of edge       | int               |


Let's take a look what kind of features we have in the `highway` column:

In [ ]:
gdf_edges['highway'].value_counts()

## Analyzing the network properties

Now as we have seen some of the basic functionalities of OSMnx such as downloading the data and converting data from graph to GeoDataFrame, we can take a look some of the analytical features of omsnx. Osmnx includes many useful functionalities to extract information about the network.

To calculate some of the basic street network measures we can use [basic_stats()](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.stats.basic_stats) function in OSMnx:

In [ ]:
# Calculate network statistics
stats = ox.basic_stats(G)
stats

## Get the centroid

For this lab, we will use the centroid of the street network as the point from which to conduct our travel isochrone maps. Note that this needs to be somewhere within the street network to build the isochrone, and does not necessarily have to be the centroid.

- [geopandas `total_bounds`](https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoSeries.total_bounds.html)

In [ ]:
# get the bounding box coordinates
minx, miny, maxx, maxy = gdf_nodes.geometry.total_bounds
print(minx)
print(miny)
print(maxx)
print(maxy)

In [ ]:
# calculate the centroid
centroid_x = (maxx-minx)/2 + minx
centroid_y = (maxy-miny)/2 + miny
print(centroid_x)
print(centroid_y)

## Get the nearest node to the centroid

Let's now find the nearest graph node (and its node ID) to the centroid point using OSMnx [distance.nearest_nodes](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.nearest_nodes). 


In [ ]:
# use osmnx's distance.nearest_nodes command to get the id for the nearest node
center_node = ox.distance.nearest_nodes(G,Y=centroid_y,X=centroid_x)
print('The id for the nearest node is ' + str(center_node))

In [ ]:
# what is this record?
gdf_nodes.loc[[center_node]]

Why `loc` and not `iloc`? The differences are confusing...

-  loc gets rows (or columns) with particular labels from the index
-  iloc gets rows (or columns) at particular positions in the index (so it only takes integers)

In this case, although the index is numeric, it does not represent a cumulative position in the dataframe, but rather is a reference to the OSM ID for that node. Therefore, we must use `loc` instead of `iloc`.

## Map the network layers

In [ ]:
# set up the subplot (single plot = ax)
fig, ax = plt.subplots(figsize=(10,10))

# add the edges to ax
gdf_edges.plot(ax=ax,
               linewidth=0.5,
               edgecolor='gray', 
               zorder=10)

# add all nodes to ax
gdf_nodes.plot(ax=ax, 
               markersize=2, 
               zorder=20)

# add the center node in red also to ax
gdf_nodes.loc[[center_node]].plot(ax=ax,
                                  color='r', 
                                  zorder=30)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron,zoom=14)

# Create isochrones

How far can you walk in 5, 10, 15, and 20 minutes from the central node? 

## Calculating travel cost

Now comes the fun part. We have thus far:

- downloaded nodes and edges for our place of interest
- reprojected them to web mercator for spatial analysis
- converted the data to geodataframes
- determined the centroid of the network
- determined the node closest to the centroid to conduct create isochrone maps

In order to create isochrone maps, we need to associate a **cost** to each edge in our network. In other words, we need an attribute that tells us exactly how long it would take an average person to traverse the distance of each edge in our network.

In [ ]:
# a quick look at the data
gdf_edges[['osmid','name','highway','length']].sample(5)

In [ ]:
# create a new column, calculate the time it takes to travel that edge
gdf_edges['walk_time'] = gdf_edges['length']/meters_per_minute

Now let's do a spot check. If a person travels 75 meters per minute, do the numbers in the new `walk_time` column make sense?

In [ ]:
gdf_edges[['osmid','name','highway','length','walk_time']].sample(10)

## Choosing colors (cmap) for our isochrones

- https://matplotlib.org/examples/color/colormaps_reference.html

OSMnx has included a super handy utility [`get_colors`](https://osmnx.readthedocs.io/en/stable/osmnx.html?highlight=get_colors#osmnx.plot.get_colors) to extract colors from existing cmap color bands. The way it works is that you ask for any number of colors (since we have 4 travel times, we want 4 colors), give it a cmap name, and it returns you a list of colors. 

In [ ]:
# assign a color hex code for each trip time isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), 
                                cmap='plasma', 
                                start=0, 
                                return_hex=True)
print(trip_times)
print(iso_colors)

In [ ]:
# reverse the sort order so that the darker color matches longer times
trip_times.sort(reverse=True)
print(trip_times)
print(iso_colors)

In [ ]:
# create a list of "zipped" time/colors
time_color = list(zip(trip_times, iso_colors))
time_color

What are zipped lists in python?

- https://towardsdatascience.com/zip-function-in-python-da91c248385d

In [ ]:
# loop through the list of time/colors
for time,color in list(time_color):
    print('The color for '+str(time)+' minutes is ' + color)

## Color each node based on travel time from point of interest

Now that we have determined the color for each travel time, let's loop through each time/color, and assign the nodes that fall within that travel time. To do so, we use a feature from [NetworkX called `ego_graph`.](https://networkx.org/documentation/stable//reference/generated/networkx.generators.ego.ego_graph.html) The ego graph determines which nodes fall within a given "radius", which in our case is determined by time.

The following loop does:

- loops through each time/color
- creates an ego graph for each time/color
- assigns the time/color in the geodataframe to each node that falls within

In [ ]:
# loop through each trip time and associated color
for time, color in list(time_color):

    # for each trip time, create an egograph of nodes that fall within that distance
    subgraph = nx.ego_graph(G, center_node, radius=time)

    print('There are ' + str(len(subgraph.nodes())) + ' nodes within ' + str(time) + ' minutes ')
    
    # for each of those nodes, update the gdf_nodes dataframe and assign it with its associated distance color
    for node in subgraph.nodes():
        gdf_nodes.loc[node,'time'] = str(time) + ' mins'
        gdf_nodes.loc[node,'color'] = color

In [ ]:
# spot check
gdf_nodes.sample(10)

In [ ]:
# the NaN values then need to be populated with a valid color
gdf_nodes['color'].fillna('#cccccc', inplace=True)

In [ ]:
# another spot check: are all values in the color column populated?
gdf_nodes.sample(10)

In [ ]:
# map it
gdf_nodes.plot(figsize=(10,10),
               color=gdf_nodes['color'],
               markersize=3)

In [ ]:
# a "full" map
# set up the subplot (single plot = ax)
fig, ax = plt.subplots(figsize=(10,10))

# add the edges to ax
gdf_edges.plot(ax=ax,
               linewidth=0.5,
               edgecolor='gray', 
               zorder=10)

# add all nodes to ax
gdf_nodes.plot(ax=ax,
               color=gdf_nodes['color'],
               markersize=4, 
               zorder=20)

# add the center node in red also to ax
gdf_nodes.loc[[center_node]].plot(ax=ax,
                                  color='r', 
                                  zorder=30)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

## Create polygons for each time area

In [ ]:
# show only the points within 5 minutes
gdf_nodes[gdf_nodes['time']=='5 mins']

In [ ]:
# put it in a variable
gdf_5 = gdf_nodes[gdf_nodes['time']=='5 mins']

In [ ]:
# plot it
gdf_5.plot(figsize=(10,10),color=gdf_5.color)

Points are great, but polygons are visually more informative. For this, we create a [convex hull](https://medium.com/@harshitsikchi/convex-hulls-explained-baab662c4e94).

![convex hull](https://miro.medium.com/max/1489/1*F4IUmOJbbLMJiTgHxpoc7Q.png)

[source](https://miro.medium.com/max/1489/1*F4IUmOJbbLMJiTgHxpoc7Q.png)

In [ ]:
# combine all the points (unary_union) and create a convex hull polygon
gdf_5.unary_union.convex_hull

## Dissolve to the rescue!

Now that we understand the process of how the isochrones will be created (from points to convex hulls), let's apply this to each of our travel times. The `dissolve` function is a handy geopandas tool that collapses and groups our data based on a given category. We can use this to create grouped points per time period.

- https://geopandas.org/aggregation_with_dissolve.html

In [ ]:
# dissolve the nodes by time
isochrones = gdf_nodes.dissolve("time")
isochrones

In [ ]:
# for each row, create a convex hull
isochrones = isochrones.convex_hull.reset_index(name='geometry')
isochrones

In [ ]:
isochrones.plot(figsize=(10,10),alpha=0.2,cmap='plasma')

# Putting it all together

In [ ]:
# set up the subplots
fig, ax = plt.subplots(figsize=(10,15))

# add the isochrones
isochrones.plot(alpha=0.4, 
                ax=ax, 
                column='time', 
                cmap='plasma', 
                legend=True,
                zorder=20)

# add the center node in red
gdf_nodes.loc[[center_node]].plot(ax=ax,color='r', zorder=30)

# add all nodes
gdf_nodes.plot(ax=ax, 
               markersize=1, 
               zorder=10)

# add the edges
gdf_edges.plot(ax=ax,
               linewidth=0.5,
               alpha=0.2,
               zorder=10)

# hide the axis
ax.axis('off')

# give it a title
ax.set_title('Walking areas from the center of ' + address,fontsize=16,pad=10)

# add the basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

# Creating "better" isochrones

![better isos](images/betterisos.png)

The material in this section has been inspired by the following sources:

- [Better Rendering of Isochrones from Network Graphs](http://kuanbutts.com/2017/12/16/osmnx-isochrones/) by Kuan Butts (Mapbox software engineer)
- [Draw an isochrone map with OSMnx](https://github.com/gboeing/osmnx-examples/blob/master/notebooks/13-isolines-isochrones.ipynb) by Geoff Boeing

In [ ]:
# additional libraries for geometry shapes
from shapely.geometry import Point, LineString, Polygon
from descartes import PolygonPatch

In [ ]:
# function to create "better" isochrones
def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

        node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({'id': list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index('id')

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union
        
        # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys

In [ ]:
# call function to create "better" isochrones for G
isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)

In [ ]:
# Create an empty geopandas GeoDataFrame
better_isos = gpd.GeoDataFrame()
better_isos['geometry'] = None

In [ ]:
# loop through the polygons and put them in a geodataframe
for i in range(len(isochrone_polys)):
    better_isos.loc[i,'geometry'] = isochrone_polys[i]
    better_isos.loc[i,'time'] =  str(trip_times[i]) + ' mins'
better_isos

In [ ]:
# create a beautiful map with all relevant layers
# set up the subplots
fig, ax = plt.subplots(figsize=(10,15))

# add the isochrones
better_isos.plot(alpha=0.4, 
                ax=ax, 
                column='time', 
                cmap='plasma', 
                edgecolor='white',
                 legend=True,
                zorder=20)

# add the center node in red
gdf_nodes.loc[[center_node]].plot(ax=ax,color='r', zorder=30)

# add all nodes
# gdf_nodes.plot(ax=ax, 
#                markersize=1, 
#                zorder=10)

# add the edges
gdf_edges.plot(ax=ax,
               linewidth=0.5,
               alpha=0.2,
               zorder=10)

# hide the axis
ax.axis('off')

# give it a title
ax.set_title('Walking areas from center of ' + place)

# add the basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter)

<div class="alert alert-info">
    
<h2>Now it's your turn!</h2>

Create isochrone maps based on a new location. Make sure not to choose a huge area (e.g. "Los Angeles" or "Tokyo" will most certainly be too large for this notebook to handle). Change parameters as you like (such as travel times), and run the notebook to produce an isochrone map.

When you are done, post your results in the [class gallery](https://docs.google.com/document/d/1u1b4r6j9Av-u3LZnzE5CL_fblss7EWnEXoSBKQ_Ao9Y/edit).

</div>